In [1]:
# move this down
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel 
ds = pd.read_csv('scotch_review.csv')

In [2]:
import requests
from bs4 import BeautifulSoup
import re
import pickle

In [3]:
URL0 = 'https://flaviar.com/bottles/scotch'

page0 = requests.get(URL0)

soup0 = BeautifulSoup(page0.text, 'lxml')

In [4]:
# scraps website for bottle edition

bottle_names = []
bottle_editions = []
bottle_types = []



class0 = soup0.find('div', {'id': 'content', 'class': 'overflow-hidden'})
class1 = class0.find('div', {'class': 'fullWidth pattern-bg sand-bg-mobile padding-top-50 padding-top-mobile-40 padding-bottom-50'})
class2 = class1.find('div', {'class': 'container'})
class3 = class2.find('div', {'class': 'row'})
class4 = class3.find('div', {'class': 'col-md-12'})
class5 = class4.find('div', {'id': 'previousboxes', 'class': 'row margin-left-0 margin-right-0 padding-left-mobile-10 padding-right-mobile-10 bottles'})
class6 = class5.find('div', {'class': 'row'})
class7 = class6.find_all('div', {'class': 'col-lg-3 col-md-4 col-sm-6'})


for i in class7:
    class8 = i.find('a', {'class': 'single-item bottle'})    
    class9 = class8.find('div', {'class': 'holder'})
    class10 = class9.find('div', {'class': 'about flex flex--space-between'})
    class11 = class10.find('div', {'class': 'desc'})
    
    bottle_name = class11.find('div', {'class': 'name'})
    bottle_edition = class11.find('div', {'class': 'edition'})
    bottle_type = class11.find('div', {'class': 'type'})
    
    bottle_names.append(bottle_name.text.replace("'","").replace(' ','-'))
    bottle_editions.append(bottle_edition.text.replace("'","").replace(' ','-'))
    bottle_types.append(bottle_type.text)
    



In [5]:
# Creates url for all editions in selected bottle type(s)
# Type choosen when URL0 is defined

url_editions = []
BOTTLE_COUNT = len(bottle_editions)
for n in range(BOTTLE_COUNT):
    url_editions.append((str('https://flaviar.com/' + bottle_names[n] + '/' + bottle_editions[n])))
    

In [6]:
print(url_editions[:4])

['https://flaviar.com/The-Naked-Grouse/The-Naked-Grouse', 'https://flaviar.com/Chivas-Regal/Chivas-Regal-12-Year-Old', 'https://flaviar.com/Dewars/Dewars-White-Label', 'https://flaviar.com/Johnnie-Walker/Johnnie-Walker-Gold-Label-Reserve']


In [21]:

appearances = []
noses = []
palates = []
finishes = []
c = 0

for url in range(len(url_editions)):
    
    c+=1
    if c == 4: break
        
    URL1 = url_editions[url]
    
    print('URL1 in url_editions:',URL1,'\n')
    
    page1 = requests.get(URL1)

    soup1 = BeautifulSoup(page1.text, 'lxml')
    
    
    
    
    profile_class0 = soup1.find('div', {'id': 'content', 'class': 'overflow-hidden'})
    profile_class1 = profile_class0.find('div', {'id': 'info', 'class': 'container'})
    profile_class2 = profile_class1.find('div', {'class': 'row shrink flex flex--stretch margin-bottom-50 margin-bottom-mobile-0'})
    profile_class3 = profile_class2.find('div', {'class': 'col-md-8 col-sm-12 col-xs-12 element info-description flex _flex-2 fit2EdgeOnMobile'})
    profile_class4 = profile_class3.find('div', {'class': 'introduce pattern-bg equal moduleTabs collapsable-content-box'})
    profile_class5 = profile_class4.find('div', {'id': 'tasting-notes','class': 'body collapse in more-body-padding'})
    profile_class6 = profile_class5.find('div', {'class': 'p-content'})
    
    
    #print(profile_class6)
    
    for p in profile_class6.find_all('div'):
        #print(profile_class6.find_all('div'))
        if (p.text == '\nAppearance / Color'):
          
            print('URL1 in profile_6:',URL1,'\n')
            
            appearances.append(str(profile_class6.div.next_sibling.next_sibling.text).strip('\r\n\t'))
            
            
            
            print(str(profile_class6.div.next_sibling.next_sibling.text).strip('\r\n\t'))

        elif (p.text == '\nNose / Aroma / Smell'):
            noses.append(str(profile_class6.div.next_sibling.next_sibling.next_sibling.
                             next_sibling.next_sibling.next_sibling.next_sibling.next_sibling.
                             text).strip('\r\n\t'))

        elif (p.text == '\nFlavor / Taste / Palate'):
            palates.append(str(profile_class6.div.next_sibling.next_sibling.next_sibling.
                               next_sibling.next_sibling.next_sibling.next_sibling.next_sibling.
                               next_sibling.next_sibling.next_sibling.next_sibling.next_sibling.
                               next_sibling.text).strip('\r\n\t')) 

        elif (p.text == '\nFinish'):
            finishes.append(str(profile_class6.div.next_sibling.next_sibling.next_sibling.
                                next_sibling.next_sibling.next_sibling.next_sibling.next_sibling.
                                next_sibling.next_sibling.next_sibling.next_sibling.next_sibling.
                                next_sibling.next_sibling.next_sibling.next_sibling.next_sibling.
                                next_sibling.next_sibling.text).strip('\r\n\t'))
        

print(appearances)
print(noses)
print(palates)
print(finishes)
    

URL1 in url_editions: https://flaviar.com/The-Naked-Grouse/The-Naked-Grouse 

URL1 in profile_6: https://flaviar.com/The-Naked-Grouse/The-Naked-Grouse 

Amber
URL1 in url_editions: https://flaviar.com/Chivas-Regal/Chivas-Regal-12-Year-Old 

URL1 in url_editions: https://flaviar.com/Dewars/Dewars-White-Label 

['Amber']
['Smooth sherry aroma followed by notes of buttered toast, malt, peach and custard.']
['Sherried palate with notes of toffee, sultanas, raspberry jam, custard and caramel.']
['Medium finish with noes of oak and cocoa.']


In [47]:
appearances = []
noses = []
palates = []
finishes = []

URL1 = url_editions[2]

page1 = requests.get(URL1)

soup1 = BeautifulSoup(page1.text, 'lxml')




profile_class0 = soup1.find('div', {'id': 'content', 'class': 'overflow-hidden'})
profile_class1 = profile_class0.find('div', {'id': 'info', 'class': 'container'})
profile_class2 = profile_class1.find('div', {'class': 'row shrink flex flex--stretch margin-bottom-50 margin-bottom-mobile-0'})
profile_class3 = profile_class2.find('div', {'class': 'col-md-8 col-sm-12 col-xs-12 element info-description flex _flex-2 fit2EdgeOnMobile'})
profile_class4 = profile_class3.find('div', {'class': 'introduce pattern-bg equal moduleTabs collapsable-content-box'})
profile_class5 = profile_class4.find('div', {'id': 'tasting-notes','class': 'body collapse in more-body-padding'})
profile_class6 = profile_class5.find('div', {'class': 'p-content'})


appearances.append(str(profile_class6.find('br').next_sibling).strip('\r\n'))

noses.append(str(profile_class6.find('br').next_sibling.next_sibling.next_sibling.next_sibling.
      next_sibling.next_sibling.next_sibling.next_sibling).strip('\r\n'))

palates.append(str(profile_class6.find('br').next_sibling.next_sibling.next_sibling.next_sibling.
      next_sibling.next_sibling.next_sibling.next_sibling.next_sibling.next_sibling.next_sibling.
      next_sibling.next_sibling.next_sibling.next_sibling).strip('\r\n'))

finishes.append(str(profile_class6.find('br').next_sibling.next_sibling.next_sibling.next_sibling.
      next_sibling.next_sibling.next_sibling.next_sibling.next_sibling.next_sibling.next_sibling.
      next_sibling.next_sibling.next_sibling.next_sibling.next_sibling.next_sibling.next_sibling.
      next_sibling.next_sibling.next_sibling.next_sibling).strip('\r\n'))





print(appearances)
print(noses)
print(palates)
print(finishes)

['Spun Hay']
['The aroma is fruit first with note of lemon zest, honey, peach, and grasses.']
['The flavor profile is a classic, 1960’s-era Scotch profile with a bit of caramel laid over some smoky peat, wild honey, and oak.']
['The finish is nicely long with lingering fresh herbs and lemon pith.\n                    ']


In [1]:
pickle.dump(profile_class6.text, open( "tasting-notes.p", "wb" ))
#print(pickle.load(open( "tasting-notes.p", "rb" )))



pickle.dump(bottle_editions, open( "bottle-editions.p", "wb" ))
#print(pickle.load(open( "bottle-editions.p", "rb" )))

NameError: name 'pickle' is not defined